## CrewAI sandbox

This notebook creates a simple CrewAI agent with a clear role, goal, tools, and a persona, then runs a quick test prompt using the `r2-env` conda environment.


In [1]:
# If not already, ensure we're in the correct env (r2-env) manually in your terminal.
# Imports
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
import os

# Configure API keys via environment variables if available
# Expecting SERPER_API_KEY and OPENAI_API_KEY to be set in r2-env
# SERPER_API_KEY = os.getenv("SERPER_API_KEY")
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# print("SERPER_API_KEY set:", bool(SERPER_API_KEY))
# print("OPENAI_API_KEY set:", bool(OPENAI_API_KEY))


In [2]:
# 1) Define agent role, goal, tools
# Web search tool (Serper)
search_tool = SerperDevTool()

research_agent = Agent(
    role="Personal Research & Writing Assistant",
    goal=(
        "Search the web for credible sources, summarize findings, and draft clear,"
        " concise outputs like emails or briefs tailored to the user's needs."
    ),
    backstory=(
        "You are a reliable and detail-oriented assistant trained to perform fast,"
        " accurate web research, synthesize information, and write well-structured"
        " summaries, outlines, and drafts. You prefer citing reputable sources."
    ),
    tools=[search_tool],
    allow_delegation=False,
    verbose=True,
)

print("Agent defined with tools:", [type(t).__name__ for t in research_agent.tools])


Agent defined with tools: ['SerperDevTool']


In [3]:
# 2) Persona reflecting the user and tasks it handles
# Edit the 'persona' variables to better match your actual background/preferences.
persona = {
    "name": "Phion's Assistant",
    "background": (
        "You reflect Phion's structured, engineering-first mindset. You favor clear"
        " reasoning, concise outputs, and reproducible steps."
    ),
    "handles": [
        "Lightweight web research with citations",
        "Summarizing articles and papers",
        "Drafting concise emails and updates",
        "Outlining study notes and project briefs",
    ],
}

persona_blurb = (
    f"You are {persona['name']}. {persona['background']} You commonly handle: "
    + ", ".join(persona["handles"]) + "."
)

# Update the agent backstory to align with persona
research_agent.backstory = persona_blurb
print(research_agent.backstory)


You are Phion's Assistant. You reflect Phion's structured, engineering-first mindset. You favor clear reasoning, concise outputs, and reproducible steps. You commonly handle: Lightweight web research with citations, Summarizing articles and papers, Drafting concise emails and updates, Outlining study notes and project briefs.


In [4]:
# 3) Simple test prompt via a Crew
intro_task = Task(
    description="Introduce yourself to the class and list what you can help with.",
    expected_output=(
        "A short paragraph introducing the agent and bullet points of capabilities."
    ),
    agent=research_agent,
)

background_task = Task(
    description="Explain Phion's background in 3 sentences based on the persona.",
    expected_output="Three succinct sentences.",
    agent=research_agent,
)

crew = Crew(
    agents=[research_agent],
    tasks=[intro_task, background_task],
    process=Process.sequential,
)

print("Crew ready.")


Crew ready.


In [5]:
# Run the crew
# Note: Requires valid OPENAI_API_KEY and SERPER_API_KEY in the environment.
results = crew.kickoff()
results


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Personal Research & Writing Assistant                                                                   │
│                                                                                                                 │
│  Task: Introduce yourself to the class and list what you can help with.                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable